# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'galaxy'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

    
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [74]:
#dados = pd.read_excel('galaxy.xlsx')
xls = pd.ExcelFile('galaxy.xlsx')
Treinamento= xls.parse('Treinamento')
Teste = xls.parse('Teste')
# Criando uma coluna nova com valores padrões (NP)
# Teste.loc[(Teste.Teste != "@@@@"),'Resultado'] = 'NP'
# Treinamento.loc[(Treinamento.Treinamento != "@@@@" ),'Resultado'] = 'NP'
# Treinamento

In [75]:
Teste_clas_palavras = []
for values in Teste.classificação:
    Teste_clas_palavras.append(values.split())

    
def limpar(frase):
    frase_final = frase.replace("."," ").replace(","," ").replace("?"," ").replace("!"," ").replace("-"," ")\
    .replace(":"," ").replace(";"," ").replace("…"," ").replace("+"," ").replace('('," ").replace("'"," ").replace('"'," ")\
    .replace(')'," ").replace("@"," ").replace("#"," ")
    return frase_final

In [115]:
base_teste = Treinamento.Treinamento
clas = Treinamento.classificação
dict_pal = dict()

for x in range (0, len(base_teste)):
    situação = clas[x]
    base_teste[x] = limpar(base_teste[x])
    twt = base_teste[x].split()
    for palavras in twt:

        if palavras in dict_pal:
            if situação == "b":
                dict_pal[palavras][0] += 1
            else:
                dict_pal[palavras][1] += 1
        else:
            if situação == "b":
                dict_pal[palavras] = [1,0] # Primeiro item da chave é a quantas vezes essa palavra é boa 
            else:
                dict_pal[palavras] = [0,1] # 2 ITEM DA CHAVE  vezes que a palavra ruim

len_dicionario = len(dict_pal)
#print(len_dicionario)

In [77]:
total_bom = 0
total_neutro = 0
B = []
n = []
for values in dict_pal:
    total_bom += dict_pal[values][0]
    total_neutro += dict_pal[values][1]
    
# print(total_bom)
# print(total_neutro)

prob_bom = total_bom/(total_bom+total_neutro)
prob_neutro = 1 - prob_bom
print("Probabilidade da palavras ser relevante",prob_bom)
print("Probabilidade da palavras não relevante",prob_neutro)

Probabilidade da palavras ser relevante 0.2150854449027696
Probabilidade da palavras não relevante 0.7849145550972304


In [114]:
base_teste = Teste.Teste
clas = Teste.classificação
dict_pal_teste = dict()
prob_palavra_rel = dict()
prob_palavra_irrel = dict()

for x in range (0, len(base_teste)):
    prob_frase_bom = 1
    prob_frase_neutro = 1

    frase = limpar(base_teste[x])
    twt = frase.split()
    
    for palavra in twt:
        if palavra in dict_pal:
            prob_palavra_rel[palavra] = (dict_pal[palavra][0] + 1)/(total_bom + len_dicionario)
            prob_palavra_irrel[palavra] = (dict_pal[palavra][1] + 1)/(total_bom + len_dicionario)
            
            prob_frase_bom *= prob_palavra_rel[palavra]
            prob_frase_neutro *= prob_palavra_irrel[palavra]
        
        else:
            prob_palavra_rel[palavra] = 1/(total_bom + len_dicionario)
            prob_palavra_irrel[palavra] = 1/(total_bom + len_dicionario)
            
            prob_frase_bom *= prob_palavra_rel[palavra]
            prob_frase_neutro *= prob_palavra_irrel[palavra]

    if prob_frase_bom > prob_frase_neutro:

        Teste.loc[(Teste.Teste == base_teste[x]),'Resultado'] = 'b'
        
    else:
        Teste.loc[(Teste.Teste == base_teste[x]),'Resultado'] = 'n'


#Teste

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [121]:
Teste.loc[(Teste.classificação == "n")& (Teste.Resultado == "n"),'Performance'] = 'NV'
Teste.loc[(Teste.classificação == "b")& (Teste.Resultado == "b"),'Performance'] = 'PV'
Teste.loc[(Teste.classificação == "n")& (Teste.Resultado == "b"),'Performance'] = 'PF'
Teste.loc[(Teste.classificação == "b")& (Teste.Resultado == "n"),'Performance'] = 'NF'

print("\n")

print(Teste.Performance.value_counts())

NV = Teste.Performance.value_counts()[0]

PV = 0 # não houveram positivos verdadeiros

NF = Teste.Performance.value_counts()[1]

PF = Teste.Performance.value_counts()[2]


Porcentagem_NV = NV/(NV+PF+NF+PV)
Porcentagem_PV = PV/(NV+PF+NF+PV)
Porcentagem_PF = PF/(NV+PF+NF+PV)
Porcentagem_NF = NF/(NV+PF+NF+PV)

print("\n")

print("Porcentagem de positivo falso:", Porcentagem_PF)
print("Porcentagem de positivo verdadeiro:", Porcentagem_PV)
print("Porcentagem de negativo verdadeiro:", Porcentagem_NV)
print("Porcentagem de negativo falso:", Porcentagem_NF)



NV    133
NF     60
PF      7
Name: Performance, dtype: int64


Porcentagem de positivo falso: 0.035
Porcentagem de positivo verdadeiro: 0.0
Porcentagem de negativo verdadeiro: 0.665
Porcentagem de negativo falso: 0.3


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


## Nossa Conclusão

A nossa conclusão foi que a perfomance do nosso classificador não foi satisfatorio pois não houve ocorrência de "Positivo Verdadeiro"  que seria a classificação da frase como bom (relevante) da classificação dos tweets no excel. A performance Negativo verdadeiro que seria ambas clasificação da base de treinamento e performance  como irrelevantes  pois ocorrreu um alto numero de acertos por parte do programas.  O quantidade de de negativo falso que é  classificado no excel de Treinamento como relevante mas foi classificado pelo programa como  irrelevante, ocorreu uma quantidade consideravel (30%) desse erro no programa. Ja a positivo falso que  é a classificação do programa para a frase como bom (relevante) mas sua classificação no excel  é irrelevante  , possuiu um pequena ocorrencia igual a 3,5%.

As mensagens com dupla negação e sarcasmo são tratadas como as outras frases . Portanto, precisariamos multiplicar as probabilidades de cada palavra da frase  de serem relevantes e irrelevantes   , e comparar a probabilidade da frase ser relevante e irrelevante obtidos para ver qual das duas é maior. Por isso , não se leva conta o sentido ou o contexto.

Eles deveriam continuar financiando o nosso projeto pois o programa conseguiu efetuar com uma alta  porcentagem  a  classificação das frase  quando elas são irrelevantes, porém o programa não conseiguiu classificar como relevantes  as frase classificadas como relevantes no excel "positivo verdadeiro". Por isso , precisariamos de financiamento para aprimorarmos esse ponto para termos um programa que consiga classificar com uma precisão satisfatoria.
